In [1]:
#Generates ratings for problems and users
#then does some analysis on it

import numpy as np
import pandas as pd
import os
import csv
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from ggplot import *
import datetime
NROWS = 10000

ModuleNotFoundError: No module named 'numpy'

In [ ]:
path = '''/w/225/1/chess/tactics/'''
os.chdir(path)
filename = '''user_tactics_problem.csv_00'''

In [ ]:


class Player:
    
    def __init__(self):
        self.rating = 1720
        self.rd = 350
        self.p = 3 * (math.log(10)**2) / ((math.pi ** 2) * 400**2)
        self.q = math.log(10)/400
        self.c = 900
    
    def getRating(self):
        return self.rating
    
    def getRd(self):
        return self.rd
    
    def f(self, oppRd):
        return 1/math.sqrt(1 + (self.p * (oppRd ** 2)))
    
    def E(self, oppRating, oppRd):
        return 1/(1 + 10**(-1 * (self.rating - oppRating) * self.f(oppRd) / 400))
    
    def K(self, oppRating, oppRd):
        k = self.q * self.f(oppRd)/ ((1/self.rd ** 2) + (self.q **2)*(self.f(oppRd)**2)*self.E(oppRating, oppRd) * (1-self.E(oppRating, oppRd)))
        if k < 16:
            return 16
        else:
            return k
    
    def update_no_games(self, timeWithoutPlaying):
        self.rd = math.sqrt(self.rd**2 + self.c * timeWithoutPlaying)
        if self.rd > 350:
            self.rd = 350
    
    def update_player(self, oppRating, oppRd, result):
        self.rating = self.rating + (self.K(oppRating, oppRd) * (int(result) - self.E(oppRating, oppRd)))
        self.rd = (math.sqrt((1/self.rd ** 2) + (self.q ** 2 ) * (self.f(oppRd) ** 2) * self.E(oppRating, oppRd) * (1-self.E(oppRating, oppRd))))** -1 

In [ ]:
"""
#step one, get "difficulty rating" for each problem, user.
#ELO code borrowed from: https://www.geeksforgeeks.org/elo-rating-algorithm/
 
# Function to calculate the Probability
def Probability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
 
# Function to calculate Elo rating
# K is a constant.
# d determines whether
# Player A wins or Player B. 
def EloRating(Ra, Rb, d):
  
    K = 30 #constant
    # To calculate the Winning
    # Probability of Player B
    Pb = Probability(Ra, Rb)
 
    # To calculate the Winning
    # Probability of Player A
    Pa = Probability(Rb, Ra)
 
    # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + K * (1 - Pa)
        Rb = Rb + K * (0 - Pb)
     
 
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        Ra = Ra + K * (0 - Pa)
        Rb = Rb + K * (1 - Pb)
    return Ra, Rb
 """

In [ ]:
#df = pd.read_csv(filename, nrows = 10)
#print(df)

userDict = {} #user_ID:Rating
problemDict = {} # ID:Rating
userGamesPlayed = {} #user_ID:number of games played
userLastPlayed = {} #user_ID:date of last game played

'''
def getRating(user, Dict, default):
    if user not in Dict:
        Dict[user] = default
    return Dict[user]    
'''
def getRating(user, Dict):
    if user not in Dict:
        Dict[user] = Player()
    return Dict[user].getRating()
    

def getDateTime(raw_dt):
    a, b, c = [int(i) for i in raw_dt.split()[0].split("-")]
    return datetime.date(a, b, c)

In [ ]:
#save ints, not floats
#write to file every 100000 lines or so to speed up
#write own 

chunksUsed = 0
newCSV = open("glicko_user_tactics_problem.csv_00", "w", newline= "")
newCSV = csv.writer(newCSV)
newCSV.writerow(["user_tactics_problem_id", "user_hash", "create_date", "date", "last_game_date", "seconds", "is_passed", "correct_move_count", "rating_change", "tactics_problem_id", "ratingUser", "varianceUser", "ratingProblem", "varianceProblem", "userGamesPlayed"])
problemProblems = []
for chunk in pd.read_csv(filename, chunksize=10000): 
    rows = []
    for index, row in chunk.iterrows():
        user = row["user_hash"]
        problem = row["tactics_problem_id"]
        
        if str(problem) in problemProblems:
            continue
        winner = row["is_passed"]
        date = getDateTime(row["create_date"])
        if user not in userLastPlayed:
            userLastPlayed[user] = date
        userRating = getRating(user, userDict)
        daysMissed = (date-userLastPlayed[user]).days
        userDict[user].update_no_games(daysMissed)
        lastGame = userLastPlayed[user]
        userLastPlayed[user] = date
        userRd = userDict[user].getRd()
        problemRating = getRating(problem, problemDict)
        problemRd = problemDict[problem].getRd()
        #userDict[user], problemDict[problem] = EloRating(userRating, problemRating, winner)
        try:
            userDict[user].update_player(problemRating, problemRd, bool(winner))
            problemDict[problem].update_player(userRating, userRd, not bool(winner))
            if user not in userGamesPlayed:
                userGamesPlayed[user] = 1
            else:
                userGamesPlayed[user] = userGamesPlayed[user] + 1

            newRow = [row["user_tactics_problem_id"], row["user_hash"], row["create_date"], str(date), str(lastGame), row["seconds"], row["is_passed"], row["correct_move_count"], row["rating_change"], row["tactics_problem_id"], str(int(userRating)), str(int(userRd)), str(int(problemRating)), str(int(problemRd)), str(userGamesPlayed[user]-1)]
            rows.append(newRow)
        except:
            print("error with rating,  user:", user)
            print("problem ID:", problem)
            problemProblems.append(str(problem))

            
        
    newCSV.writerows(rows)    
    chunksUsed += 1
    if chunksUsed % 10 == 0:
        print(chunksUsed)

In [ ]:
'''newCSV = open("glicko_user_tactics_problem.csv_00", "r").readlines()
for line in newCSV:
    print(line)
'''
#%store -r problemProblems




In [ ]:
len(userDict)
print(max(userGamesPlayed.values()))
'''import operator
sorted_x = sorted(userDict.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x'''

In [ ]:
'''%store -r userDict
%store -r problemDict
%store -r userGamesPlayed'''

#now we have our variables: difficulty (problemsDict), skill (userDict), and time (time)

In [ ]:
plt.hist([p.getRating() for p in userDict.values()], [500 + (i*50) for i in range(40)])
#most people are clumped just below 1500, some are just above.
print("mean rating is ", sum([p.getRating() for p in userDict.values()])/len(userDict))

In [ ]:
plt.hist([p.getRd() for p in userDict.values()], [0 + (i*10) for i in range(50)])



In [ ]:
plt.hist(list(userGamesPlayed.values()), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]) 
#probably because most people have under 5 puzzles completed

In [ ]:
plt.hist([p.getRating() for p in problemDict.values()], [500 + (i*50) for i in range(40)])

print("mean rating is ", sum([p.getRating() for p in problemDict.values()])/len(problemDict))

In [ ]:
#Another thing to consider plotting: ELO by number of moves to solve - although not sure if that's relevant to the study

In [ ]:
df = pd.read_csv(filename, nrows = NROWS)

In [ ]:
sum(df["is_passed"] == 1)/len(df.index) #percentage solved

In [ ]:
userRatingData = pd.DataFrame([(p, userDict[p].getRating(), userDict[p].getRd()) for p in userDict],
                      columns=['userHash','ratingUser', 'varianceUser'])
problemRatingData = pd.DataFrame([(p, problemDict[p].getRating(), problemDict[p].getRd()) for p in problemDict],
                      columns=['problemID','ratingProblem', 'varianceProblem'])

df2 = pd.merge(df, userRatingData, left_on='user_hash', right_on= "userHash", how="left")
df2 = pd.merge(df2, problemRatingData, left_on='tactics_problem_id', right_on= "problemID", how="left")


In [ ]:
df2.drop(columns = ["rating_change","userHash", "problemID" ])
df2[1:5]


In [ ]:
#%store -r df2

In [ ]:
trainingSet = df2[:int(NROWS*.8)]
testingSet = df2[int(NROWS*.8):]

In [ ]:
rf = RandomForestClassifier(n_estimators = 20, max_depth = 40)
X = trainingSet[['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem']]
y = trainingSet['is_passed']
rf.fit(X,y)

In [ ]:
Xtest = testingSet[['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem']]
ytest = testingSet['is_passed']
prediction = lr.predict(Xtest)
print("all Variable RF model accuracy: ", accuracy_score(ytest, prediction))

In [ ]:
lr = LogisticRegression()
X = trainingSet[['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem']]
y = trainingSet['is_passed']
lr.fit(X, y)

In [ ]:
lr.coef_

In [ ]:
Xtest = testingSet[['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem']]
ytest = testingSet['is_passed']
prediction = lr.predict(Xtest)
print("all Variable Logistic model accuracy: ", accuracy_score(ytest, prediction))



In [ ]:
from collections import deque
variableList = deque(['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem'])


for i in range(len(variableList)):
    varMissing = variableList.popleft()
    lr2 = LogisticRegression()
    X = trainingSet[list(variableList)]
    y = trainingSet['is_passed']
    lr2.fit(X, y)
    Xtest = testingSet[list(variableList)]
    prediction = lr2.predict(Xtest)
    print("No", varMissing, "model accuracy: ", accuracy_score(ytest, prediction))
    variableList.append(varMissing)
'''
lr2 = LogisticRegression()
X = trainingSet[['ratingUser', 'ratingProblem']]
y = trainingSet['is_passed']
lr2.fit(X, y)
Xtest = testingSet[['ratingUser', 'ratingProblem']]
prediction = lr2.predict(Xtest)
print("No Time model accuracy: ", accuracy_score(ytest, prediction))

lr3 = LogisticRegression()
X = trainingSet[['seconds', 'ratingProblem']]
y = trainingSet['is_passed']
lr3.fit(X, y)
Xtest = testingSet[['seconds', 'ratingProblem']]
prediction = lr3.predict(Xtest)
print("all User Rating Model accuracy: ", accuracy_score(ytest, prediction))


lr4 = LogisticRegression()
X = trainingSet[['seconds', 'ratingUser']]
y = trainingSet['is_passed']
lr4.fit(X, y)
Xtest = testingSet[['seconds', 'ratingUser']]
prediction = lr4.predict(Xtest)
print("all Problem Rating Model accuracy: ", accuracy_score(ytest, prediction))'''


In [ ]:
X = trainingSet[['seconds', 'ratingUser', 'ratingProblem', 'varianceUser', 'varianceProblem']]
logit = sm.Logit(y, X)
result = logit.fit()

In [ ]:
print(result.summary())

In [ ]:
plt.hist(list(df2["seconds"]), list(range(80)))

In [ ]:
x = 0
ratingToAccUser = {}
ratingToAccProblem = {}
for index, line in df2.iterrows():
    x += 1
    ratingU = int(line['ratingUser'])
    ratingP = int(line['ratingProblem'])
    if ratingU not in ratingToAccUser:
        ratingToAccUser[ratingU] = [0,0]
    if ratingP not in ratingToAccProblem:
        ratingToAccProblem[ratingP] = [0,0]
    ratingToAccUser[ratingU][1] += 1
    ratingToAccProblem[ratingP][1] += 1
    ratingToAccProblem[ratingP][0] += 1
    if line['is_passed']:
        ratingToAccUser[ratingU][0]+= 1
        ratingToAccProblem[ratingP][0] -= 1
    if x %50000 == 0:
        print(x/len(df2))

        
        
    

In [ ]:
#df2[""]
#test = df2.groupby("ratingUser")["is_passed"].mean()
#test

In [ ]:
fullRatingToAccUser1 = [(key, ratingToAccUser[key][0]/ratingToAccUser[key][1]) for key in ratingToAccUser]
fullRatingToAccProblem1 = [(key, ratingToAccProblem[key][0]/ratingToAccProblem[key][1]) for key in ratingToAccProblem]

In [ ]:
frtau = pd.DataFrame(fullRatingToAccUser1, columns=['rating','accuracy'])
frtap = pd.DataFrame(fullRatingToAccProblem1, columns=['rating','accuracy'])
frtap = frtap.sort_values("rating")
frtau = frtau.sort_values("rating")
frtap

In [ ]:

plt.plot('rating', 'accuracy', data = frtau)
plt.plot('rating', 'accuracy', data = frtap)